In [2]:
# MLee; dask is updated to ver.0.17.1 using Anaconda Navigator to solve following error: 
#       {AttributeError: module 'pandas.core.computation' has no attribute 'expressions'}

from __future__ import print_function
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from scnn_model import SCNN_MODEL
from tensorflow.contrib import learn
from six.moves import cPickle as pickle

In [3]:
# Load vocabulary and the word2vec model
pickle_file = 'save.pickle'
with open(pickle_file, 'rb') as f :
    save = pickle.load(f)
    wordsVectors = save['wordsVectors']
    vocabulary = save['vocabulary']
    del save  # hint to help gc free up memory
print('Vocabulary and the word2vec loaded')
print('Vocabulary size is ', len(vocabulary))
print('Word2Vec model shape is ', wordsVectors.shape)

#Load training data, training labels, validation data, validation labels
pickle_file = 'data_saved.pickle'
with open(pickle_file, 'rb') as f :
    save = pickle.load(f)
    train_data = save['train_data']
    train_labels = save['train_labels']
    validation_data = save['validation_data']
    validation_labels = save['validation_labels']
    del save  # hint to help gc free up memory
print('train data shape ', train_data.shape)
print('train labels shape ', train_labels.shape)
print('validation data shape ', validation_data.shape)
print('validation labels shape ', validation_labels.shape)

Vocabulary and the word2vec loaded
Vocabulary size is  9687
Word2Vec model shape is  (9687, 300)
train data shape  (1280, 160)
train labels shape  (1280, 2)
validation data shape  (160, 160)
validation labels shape  (160, 2)


In [4]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    Original taken from https://github.com/dennybritz/cnn-text-classification-tf/blob/master/data_helpers.py
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [5]:
# Model Hyperparameters
SENTENCE_PER_REVIEW = 16
WORDS_PER_SENTENCE = 10
EMBEDDING_DIM = 300
FILTER_WIDTHS_SENT_CONV = np.array([3, 4, 5])
NUM_FILTERS_SENT_CONV = 100
FILTER_WIDTHS_DOC_CONV = np.array([3, 4, 5])
NUM_FILTERS_DOC_CONV = 100
NUM_CLASSES = 2
DROPOUT_KEEP_PROB = 0.5
L2_REG_LAMBDA = 0.0
BATCH_SIZE = 64
NUM_EPOCHS = 100
EVALUATE_EVERY = 100   # Evaluate model on the validation set after 100 steps
CHECKPOINT_EVERY = 100 # Save model after each 200 steps
NUM_CHECKPOINTS = 5    # Keep only the 5 most recents checkpoints
LEARNING_RATE = 1e-3   # The learning rate

In [7]:
'''Training the model'''
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = SCNN_MODEL(sentence_per_review=SENTENCE_PER_REVIEW, 
                        words_per_sentence=WORDS_PER_SENTENCE, 
                        wordVectors=wordsVectors, 
                        embedding_size=EMBEDDING_DIM, 
                        filter_widths_sent_conv=FILTER_WIDTHS_SENT_CONV, 
                        num_filters_sent_conv=NUM_FILTERS_SENT_CONV, 
                        filter_widths_doc_conv=FILTER_WIDTHS_DOC_CONV, 
                        num_filters_doc_conv=NUM_FILTERS_DOC_CONV, 
                        num_classes=NUM_CLASSES, 
                        l2_reg_lambda=L2_REG_LAMBDA)
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))
        
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        
        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=NUM_CHECKPOINTS)
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.input_size: len(y_batch),
                cnn.dropout: DROPOUT_KEEP_PROB
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
        
        
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.input_size: y_batch.shape[0],
                cnn.dropout: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        print('train data shape ', train_data.shape)
        print('train labels shape ', train_labels.shape)
        print('validation data shape ', validation_data.shape)
        print('validation labels shape ', validation_labels.shape)
        
        # Generate batches
        batches = batch_iter(
            list(zip(train_data, train_labels)), BATCH_SIZE, NUM_EPOCHS)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % EVALUATE_EVERY == 0:
                print("\nEvaluation:")
                dev_step(validation_data, validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % CHECKPOINT_EVERY == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Writing to C:\Users\ML\Documents\Python Scripts\cnn-spam-review-detection-tf-master\runs\1519769974

train data shape  (1280, 160)
train labels shape  (1280, 2)
validation data shape  (160, 160)
validation labels shape  (160, 2)
2018-02-27T16:19:37.901638: step 1, loss 1.13207, acc 0.515625
2018-02-27T16:19:38.619116: step 2, loss 1.18641, acc 0.421875
2018-02-27T16:19:39.320548: step 3, loss 0.890996, acc 0.578125
2018-02-27T16:19:40.051852: step 4, loss 1.00729, acc 0.5
2018-02-27T16:19:40.769550: step 5, loss 1.0189, acc 0.484375
2018-02-27T16:19:41.498497: step 6, loss 1.07295, acc 0.5
2018-02-27T16:19:42.218315: step 7, loss 1.10024, acc 0.546875
2018-02-27T16:19:42.974582: step 8, loss 0.954503, acc 0.515625
2018-02-27T16:19:43.732888: step 9, loss 0.856246, acc 0.59375
2018-02-27T16:19:44.610978: step 10, loss 1.28536, acc 0.421875
2018-02-27T16:19:45.474273: step 11, loss 1.04525, acc 0.59375
2018-02-27T16:19:46.199959: step 12, loss 1.01977, acc 0.484375
2018-02-27T16:19:46.95

2018-02-27T16:21:09.770283: step 123, loss 0.875316, acc 0.515625
2018-02-27T16:21:10.494207: step 124, loss 0.951078, acc 0.4375
2018-02-27T16:21:11.215124: step 125, loss 0.863465, acc 0.515625
2018-02-27T16:21:11.942059: step 126, loss 0.773966, acc 0.53125
2018-02-27T16:21:12.656959: step 127, loss 0.894639, acc 0.53125
2018-02-27T16:21:13.378879: step 128, loss 0.85657, acc 0.609375
2018-02-27T16:21:14.110826: step 129, loss 0.719865, acc 0.59375
2018-02-27T16:21:14.832746: step 130, loss 0.854524, acc 0.546875
2018-02-27T16:21:15.564692: step 131, loss 0.801865, acc 0.625
2018-02-27T16:21:16.273578: step 132, loss 0.884934, acc 0.5
2018-02-27T16:21:17.001514: step 133, loss 0.841848, acc 0.59375
2018-02-27T16:21:17.727445: step 134, loss 0.697716, acc 0.609375
2018-02-27T16:21:18.453376: step 135, loss 0.775621, acc 0.59375
2018-02-27T16:21:19.176300: step 136, loss 0.811563, acc 0.484375
2018-02-27T16:21:19.974422: step 137, loss 0.813715, acc 0.53125
2018-02-27T16:21:20.831703:

2018-02-27T16:22:42.032257: step 246, loss 0.699646, acc 0.625
2018-02-27T16:22:42.749018: step 247, loss 0.713023, acc 0.609375
2018-02-27T16:22:43.464998: step 248, loss 0.601228, acc 0.6875
2018-02-27T16:22:44.197187: step 249, loss 0.562956, acc 0.75
2018-02-27T16:22:44.916264: step 250, loss 0.564509, acc 0.734375
2018-02-27T16:22:45.646225: step 251, loss 0.626238, acc 0.671875
2018-02-27T16:22:46.363744: step 252, loss 0.590962, acc 0.75
2018-02-27T16:22:47.092835: step 253, loss 0.554945, acc 0.78125
2018-02-27T16:22:47.816762: step 254, loss 0.569838, acc 0.75
2018-02-27T16:22:48.525646: step 255, loss 0.503118, acc 0.65625
2018-02-27T16:22:49.249572: step 256, loss 0.545078, acc 0.765625
2018-02-27T16:22:49.979513: step 257, loss 0.561027, acc 0.8125
2018-02-27T16:22:50.708452: step 258, loss 0.509522, acc 0.671875
2018-02-27T16:22:51.424358: step 259, loss 0.581766, acc 0.671875
2018-02-27T16:22:52.140261: step 260, loss 0.419078, acc 0.828125
2018-02-27T16:22:52.869775: ste

2018-02-27T16:24:14.789982: step 369, loss 0.320465, acc 0.84375
2018-02-27T16:24:15.511226: step 370, loss 0.18851, acc 0.9375
2018-02-27T16:24:16.377446: step 371, loss 0.268012, acc 0.859375
2018-02-27T16:24:17.215943: step 372, loss 0.215404, acc 0.953125
2018-02-27T16:24:17.968134: step 373, loss 0.204245, acc 0.90625
2018-02-27T16:24:18.718167: step 374, loss 0.214635, acc 0.921875
2018-02-27T16:24:19.462146: step 375, loss 0.261803, acc 0.90625
2018-02-27T16:24:20.200109: step 376, loss 0.264303, acc 0.875
2018-02-27T16:24:20.933059: step 377, loss 0.173966, acc 0.984375
2018-02-27T16:24:21.663000: step 378, loss 0.217549, acc 0.875
2018-02-27T16:24:22.396952: step 379, loss 0.203553, acc 0.953125
2018-02-27T16:24:23.116866: step 380, loss 0.295462, acc 0.875
2018-02-27T16:24:23.870874: step 381, loss 0.234273, acc 0.9375
2018-02-27T16:24:24.596805: step 382, loss 0.191309, acc 0.890625
2018-02-27T16:24:25.326745: step 383, loss 0.123961, acc 0.984375
2018-02-27T16:24:26.063705:

2018-02-27T16:25:48.592259: step 493, loss 0.0728695, acc 0.96875
2018-02-27T16:25:49.318190: step 494, loss 0.0822248, acc 0.96875
2018-02-27T16:25:50.047129: step 495, loss 0.0800834, acc 0.96875
2018-02-27T16:25:50.764037: step 496, loss 0.0932444, acc 0.953125
2018-02-27T16:25:51.483950: step 497, loss 0.0626095, acc 0.96875
2018-02-27T16:25:52.346244: step 498, loss 0.123502, acc 0.953125
2018-02-27T16:25:53.217561: step 499, loss 0.0523176, acc 1
2018-02-27T16:25:53.953518: step 500, loss 0.0830441, acc 0.96875

Evaluation:
2018-02-27T16:25:54.524037: step 500, loss 0.408136, acc 0.825

Saved model checkpoint to C:\Users\ML\Documents\Python Scripts\cnn-spam-review-detection-tf-master\runs\1519769974\checkpoints\model-500

2018-02-27T16:25:56.047661: step 501, loss 0.0575412, acc 1
2018-02-27T16:25:56.785623: step 502, loss 0.0721822, acc 0.96875
2018-02-27T16:25:57.513559: step 503, loss 0.0993012, acc 0.96875
2018-02-27T16:25:58.239491: step 504, loss 0.0415418, acc 0.984375
201

...

2018-02-27T16:37:06.510782: step 1390, loss 0.00763449, acc 1
2018-02-27T16:37:07.230698: step 1391, loss 0.00161074, acc 1
2018-02-27T16:37:07.991722: step 1392, loss 0.000857222, acc 1
2018-02-27T16:37:08.746754: step 1393, loss 0.000892207, acc 1
2018-02-27T16:37:09.478701: step 1394, loss 0.00231635, acc 1
2018-02-27T16:37:10.220675: step 1395, loss 0.000615066, acc 1
2018-02-27T16:37:10.945608: step 1396, loss 0.00396849, acc 1
2018-02-27T16:37:11.669528: step 1397, loss 0.00180999, acc 1
2018-02-27T16:37:12.388442: step 1398, loss 0.00114543, acc 1
2018-02-27T16:37:13.103343: step 1399, loss 0.00249368, acc 1
2018-02-27T16:37:13.830276: step 1400, loss 0.000341373, acc 1

Evaluation:
2018-02-27T16:37:14.395780: step 1400, loss 0.625047, acc 0.8

Saved model checkpoint to C:\Users\ML\Documents\Python Scripts\cnn-spam-review-detection-tf-master\runs\1519769974\checkpoints\model-1400

2018-02-27T16:37:15.852925: step 1401, loss 0.00248449, acc 1
2018-02-27T16:37:16.562813: step 1402

2018-02-27T16:38:41.450641: step 1515, loss 0.000660722, acc 1
2018-02-27T16:38:42.175074: step 1516, loss 0.000584727, acc 1
2018-02-27T16:38:42.874768: step 1517, loss 0.00106565, acc 1
2018-02-27T16:38:43.589448: step 1518, loss 0.00226172, acc 1
2018-02-27T16:38:44.322631: step 1519, loss 0.00046918, acc 1
2018-02-27T16:38:45.040489: step 1520, loss 0.0010979, acc 1
2018-02-27T16:38:45.756812: step 1521, loss 0.00192799, acc 1
2018-02-27T16:38:46.473804: step 1522, loss 0.00411934, acc 1
2018-02-27T16:38:47.196607: step 1523, loss 0.0030582, acc 1
2018-02-27T16:38:47.911711: step 1524, loss 0.000457783, acc 1
2018-02-27T16:38:48.639475: step 1525, loss 0.000848305, acc 1
2018-02-27T16:38:49.356522: step 1526, loss 0.00273527, acc 1
2018-02-27T16:38:50.087830: step 1527, loss 0.000305732, acc 1
2018-02-27T16:38:50.806012: step 1528, loss 0.000916095, acc 1
2018-02-27T16:38:51.523049: step 1529, loss 0.000327964, acc 1
2018-02-27T16:38:52.238156: step 1530, loss 0.00248348, acc 1
201

2018-02-27T16:40:17.004850: step 1643, loss 0.00122571, acc 1
2018-02-27T16:40:17.797783: step 1644, loss 0.00110779, acc 1
2018-02-27T16:40:18.513328: step 1645, loss 0.00537211, acc 1
2018-02-27T16:40:19.242593: step 1646, loss 0.00452232, acc 1
2018-02-27T16:40:19.977080: step 1647, loss 0.000728842, acc 1
2018-02-27T16:40:20.696066: step 1648, loss 0.000428557, acc 1
2018-02-27T16:40:21.412960: step 1649, loss 0.000964916, acc 1
2018-02-27T16:40:22.129141: step 1650, loss 0.000449137, acc 1
2018-02-27T16:40:22.862212: step 1651, loss 0.00075889, acc 1
2018-02-27T16:40:23.610542: step 1652, loss 0.000428978, acc 1
2018-02-27T16:40:24.328982: step 1653, loss 0.000506276, acc 1
2018-02-27T16:40:25.056268: step 1654, loss 0.000742547, acc 1
2018-02-27T16:40:25.778829: step 1655, loss 0.00190677, acc 1
2018-02-27T16:40:26.518459: step 1656, loss 0.000625437, acc 1
2018-02-27T16:40:27.245606: step 1657, loss 0.000368861, acc 1
2018-02-27T16:40:27.965575: step 1658, loss 0.00583655, acc 1

2018-02-27T16:41:53.677718: step 1771, loss 0.00172799, acc 1
2018-02-27T16:41:54.420693: step 1772, loss 0.00155576, acc 1
2018-02-27T16:41:55.153643: step 1773, loss 0.0200574, acc 1
2018-02-27T16:41:55.901632: step 1774, loss 0.0146576, acc 0.984375
2018-02-27T16:41:56.636586: step 1775, loss 0.000583705, acc 1
2018-02-27T16:41:57.372544: step 1776, loss 0.000324952, acc 1
2018-02-27T16:41:58.102486: step 1777, loss 0.00149965, acc 1
2018-02-27T16:41:58.836438: step 1778, loss 0.0233841, acc 0.984375
2018-02-27T16:41:59.570391: step 1779, loss 0.0089104, acc 1
2018-02-27T16:42:00.303340: step 1780, loss 0.000423496, acc 1
2018-02-27T16:42:01.026703: step 1781, loss 0.000955023, acc 1
2018-02-27T16:42:01.766257: step 1782, loss 0.000600939, acc 1
2018-02-27T16:42:02.499704: step 1783, loss 0.00271372, acc 1
2018-02-27T16:42:03.234023: step 1784, loss 0.000610847, acc 1
2018-02-27T16:42:03.966847: step 1785, loss 0.0129484, acc 0.984375
2018-02-27T16:42:04.716241: step 1786, loss 0.00

2018-02-27T16:43:30.741566: step 1899, loss 0.00665618, acc 1
2018-02-27T16:43:31.489407: step 1900, loss 0.000205688, acc 1

Evaluation:
2018-02-27T16:43:32.066352: step 1900, loss 0.78933, acc 0.825

Saved model checkpoint to C:\Users\ML\Documents\Python Scripts\cnn-spam-review-detection-tf-master\runs\1519769974\checkpoints\model-1900

2018-02-27T16:43:33.660327: step 1901, loss 0.00367161, acc 1
2018-02-27T16:43:34.360220: step 1902, loss 0.000755122, acc 1
2018-02-27T16:43:35.089783: step 1903, loss 0.0179512, acc 0.984375
2018-02-27T16:43:35.821348: step 1904, loss 0.0004333, acc 1
2018-02-27T16:43:36.540007: step 1905, loss 0.00232745, acc 1
2018-02-27T16:43:37.256557: step 1906, loss 0.00124728, acc 1
2018-02-27T16:43:38.000410: step 1907, loss 0.000250635, acc 1
2018-02-27T16:43:38.717577: step 1908, loss 0.00213735, acc 1
2018-02-27T16:43:39.439226: step 1909, loss 0.00154139, acc 1
2018-02-27T16:43:40.171742: step 1910, loss 0.00218059, acc 1
2018-02-27T16:43:40.906491: step